In [1]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '../core'))
import kaggle_support as kgs
import importlib
import matplotlib.pyplot as plt
importlib.reload(kgs)
import numpy as np
import cupy as cp
from dataclasses import dataclass, field, fields
import pack_cuda
import pack_vis
import pack_cost
import copy
import time
import pack_dynamics
from IPython.display import HTML, display, clear_output
pack_cuda.USE_FLOAT32 = True
pack_cuda._ensure_initialized()


local
local
local
Preallocate for cost


In [2]:
gen = np.random.default_rng(seed=12345)
runner = pack_dynamics.Optimizer()
runner.n_iterations *= 2
size_setup_scaled = 0.65
n_ensembles = 10
n_trees = 200
xyt = np.random.default_rng(seed=52).uniform(-0.5, 0.5, size=(n_ensembles, n_trees, 3))
xyt = xyt * [[[size_setup_scaled, size_setup_scaled, np.pi]]]
xyt = np.array(xyt, dtype=kgs.dtype_np)
h = np.array([[[2*size_setup_scaled,0.,0.]]*n_ensembles], dtype=kgs.dtype_np)
h = h[0]
sol = kgs.SolutionCollectionSquare()
sol.xyt = cp.array(xyt, dtype=kgs.dtype_cp)
sol.h = cp.array(h, dtype=kgs.dtype_cp)
print(sol.xyt.shape, sol.h.shape)
sol.check_constraints()
N=4
streams = [cp.cuda.Stream(non_blocking=True) for _ in range(N)]
runners = [copy.deepcopy(runner) for _ in range(N)]
sols = [copy.deepcopy(sol) for _ in range(N)]


(10, 200, 3) (10, 3)


In [3]:
%%time
runners[0].run_simulation(sols[0]);
None

CPU times: user 5.25 s, sys: 360 ms, total: 5.61 s
Wall time: 5.74 s


In [4]:
%%time
for i in range(N):
    runners[i].run_simulation(sols[i])

CPU times: user 19.9 s, sys: 1.59 s, total: 21.5 s
Wall time: 21.2 s


In [5]:
%%time
for i in range(N):
    with streams[i]:
        runners[i].run_simulation(sols[i])
#streams[i].synchronize()

CPU times: user 15.2 s, sys: 1.42 s, total: 16.6 s
Wall time: 16.3 s
